In [1]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [18]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Read Document

In [24]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a8.input'
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a8.key'

In [25]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [26]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [27]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)

### Get Coref for threshold match by NP

In [28]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)

### Get Coref for Word Embedding Similarity by NP

In [29]:
coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [30]:
coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging coref_dict_match_NP with coref_dict_all_sorted_top3

In [31]:
def mergeDicts(string_matching_dict, word_embedding_dict):
    
    d1 = string_matching_dict.copy()
    d2 = word_embedding_dict.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return d1

In [32]:
coref_final = mergeDicts(coref_dict_match_NP, coref_dict_all_sorted)

In [33]:
#coref_final

### Passing the reference dict to Hobbs

In [34]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [35]:
# coref_final_with_pro

In [36]:
for cluster_id, cluster_val in cluster_head_dict.items():
    if(cluster_id not in coref_final_with_pro.keys()):
        coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]

### Removing the Determiners from the start

In [37]:
for cluster in coref_final_with_pro.keys():
    for i in range(0, len(coref_final_with_pro[cluster])):
        if(len(coref_final_with_pro[cluster][i][2].split(' '))>1):
            ref_word_list = coref_final_with_pro[cluster][i][2].strip().split(' ')
            check = pos_tag(ref_word_list)
            if len(check) >1:
                if 'DT' in check[0][1]:
                    sentence = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
                    coref_final_with_pro[cluster][i][2] = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
                    #print(coref_final_with_pro[cluster][i])
# coref_final_with_pro

### Print Output

In [38]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [39]:
%%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

In [40]:
with open('a8.response','w') as f:
    f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans